# Creating an Index

## Setup

In [ ]:
import os
import time
import pandas as pd
from pinecone import Pinecone
from dotenv import load_dotenv
from pinecone import ServerlessSpec

load_dotenv()

## configure client

In [ ]:
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

In [ ]:
cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

## Giving our index a name

In [ ]:
index_name = "lab-29"

## Delete the index, if an index of the same name already exists

In [ ]:
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

## Creating a Pinecone Index.

In [ ]:
dimensions = 3
pc.create_index(
    name=index_name,
    dimension=dimensions,
    metric="cosine",
    spec=spec
)

# wait for index to be ready before connecting
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

In [ ]:
index = pc.Index(index_name)

In [ ]:
df = pd.DataFrame(
    data={
        "id": ["A", "B"],
        "vector": [[1., 1., 1.], [1., 2., 3.]]
    })
df

In [ ]:
index.upsert(vectors=zip(df.id, df.vector))

In [ ]:
index.describe_index_stats()

In [ ]:
index.query(
    vector=[2., 2., 2.],
    top_k=5,
    include_values=True) 

In [ ]:
pc.delete_index(index_name)